# Evaluate the RAG system

To evaluate the LLM answer we will follow the below steps:
- In the ground truth dataset we have a user question along with the actual answer of this question from the FAQ document
- Pass each question of the ground truth dataset to the RAG sytem to get the LLM answer of that question
- Create the embedding of the actual answer and the LLM Answer
- Compute the cosine similarity and other metrics of the two answer to evaluate the RAG system

In [4]:
import requests 
import pandas as pd
from openai import OpenAI
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
import os

In [5]:
# Load environment variables from .envrc
load_dotenv(".envrc")

True

## Load all the necessary documents

In [6]:
# Load the FAQ records with their ids
# Create the github url
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
# Request the json data
docs_response = requests.get(docs_url)
# Open the json data
documents = docs_response.json()
# Verify the first record
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [24]:
# Create a dictionary with any id containing the full record
doc_idx = {d['id']: d for d in documents}

In [17]:
# Load the ground truth dataset
df_ground_truth = pd.read_csv('ground-truth-data.csv')
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

## Index the necessary documents

In [8]:
# Initialize the client 
es_client = Elasticsearch('http://localhost:9200') # This is the port created after running the docker file

# Create the Schema of the Elastic Search Index
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,            # Here we are using the dimensionality of the embedding we want to store 
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

# Provide the name of the index
index_name = "vector_questions"
# Create the elastic search index
response = es_client.indices.create(index=index_name, body=index_settings)
# Verify that elastic search is created
response

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'vector_questions'})

In [9]:
# Initialize the selected model to create the embeddings
model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")

# Create the embeddings for each record in our FAQ dataset
for doc in tqdm(documents):
    # Extract the text fields you want to embed
    question_text = doc['question'] + ' ' + doc['text']
    # Create the embedding for each text field
    doc['question_text_vector'] = model.encode(question_text)

/Users/georgestavrakis/opt/anaconda3/envs/LLMZoomcamp/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


  0%|          | 0/948 [00:00<?, ?it/s]

In [10]:
# Fit the documents into the elastic search index
for doc in tqdm(documents):
    es_client.index(index = index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Create the RAG system

In [11]:
# Create the retrieval function for the elastic search
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

# Create the retrieval part of rag with the optimal results
def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

In [12]:
# Create the function to build the prompt
def build_prompt(query, search_results):
    # Create the prompt template
    prompt_template = '''
    You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    '''.strip()
    
    # Create the context from the search results
    context = ''
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    # Putting the context and query all together
    prompt = prompt_template.format(question = query, context = context).strip()
    return prompt

In [13]:
# Initialize the openai instance
client = OpenAI()

def llm(prompt, model='gpt-4o'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [14]:
# Create the rag system function
def rag(query: dict, model='gpt-4o') -> str:
    # Search the knowledge base
    search_results = question_text_vector_knn(query)
    # Prepare the prompt for the model
    prompt = build_prompt(query['question'], search_results)
    # Use the llm to get the response
    answer = llm(prompt, model=model)
    return answer

## Create LLM answers based on ground truth questions

In [22]:
# Take the first 5 entries of the ground truth dataset
sample = ground_truth[:5]
# Initialize the answers
answers = {}


In [26]:
# Use the RAG system to generate the answers from the sample with gpt-4o
for i, rec in enumerate(tqdm(sample)):
    # Create a cache so in case tha the rag breaks we don't need to rerun it
    if i in answers:
        continue
    # Generate the LLM answer
    answer_llm = rag(rec)
    # Extract the document Id from the ground truth
    doc_id = rec['document']
    # Extract the original record from the ground truth
    original_doc = doc_idx[doc_id]
    # Save the answer from the ground truth record
    answer_orig = original_doc['text']
    # Save the answers in 1 entry
    answers[i] = {
        'answer_llm': answer_llm,
        'answer_orig': answer_orig,
        'document': doc_id,
        'question': rec['question'],
        'course': rec['course']
    }

  0%|          | 0/5 [00:00<?, ?it/s]

In [30]:
# View the results as a Dataframe
pd.DataFrame(answers.values())

,answer_llm,answer_orig,document,question,course
0,NONE,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up for the course using the link ...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,NONE,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,NONE,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [35]:
# Initialize the answers
answers_35 = {}

In [36]:
# Use the RAG system to generate the answers from the sample with Gpt 3.5 turbo
for i, rec in enumerate(tqdm(sample)):
    # Create a cache so in case tha the rag breaks we don't need to rerun it
    if i in answers_35:
        continue
    # Generate the LLM answer
    answer_llm = rag(rec, model='gpt-3.5-turbo')
    # Extract the document Id from the ground truth
    doc_id = rec['document']
    # Extract the original record from the ground truth
    original_doc = doc_idx[doc_id]
    # Save the answer from the ground truth record
    answer_orig = original_doc['text']
    # Save the answers in 1 entry
    answers_35[i] = {
        'answer_llm': answer_llm,
        'answer_orig': answer_orig,
        'document': doc_id,
        'question': rec['question'],
        'course': rec['course']
    }

  0%|          | 0/5 [00:00<?, ?it/s]

In [37]:
# View the results as a Dataframe
pd.DataFrame(answers_35.values())

,answer_llm,answer_orig,document,question,course
0,NONE,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,NONE,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,NONE,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,NONE,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,You can structure your questions and answers f...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [34]:
answers_35

{}